<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark

%pip install pyspark
%pip install findspark

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark
from pyspark.sql import SparkSession

findspark.init()

In [3]:
# Start session

spark = SparkSession.builder.appName('Search Term Analysis').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/12/11 21:12:17 WARN Utils: Your hostname, fcode resolves to a loopback address: 127.0.1.1; using 172.27.243.144 instead (on interface eth0)
23/12/11 21:12:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/11 21:12:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/11 21:12:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [ ]:
!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv'

In [5]:
# Load the csv into a spark dataframe

search_df = spark.read.csv('searchterms.csv', header=True, inferSchema=True)

In [6]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

num_rows = search_df.count()
num_cols = len(search_df.columns)

shape = (num_rows, num_cols)

print('Number of rows=', num_rows)
print('Number of columns=', num_cols)
print('Shape=', shape)

Number of rows= 10000
Number of columns= 4
Shape= (10000, 4)


In [7]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

search_df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [8]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

search_df.schema['searchterm'].dataType
# ============== OR ==================
dict(search_df.dtypes)['searchterm']

'string'

In [9]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

search_df.filter(search_df['searchterm'] == 'gaming laptop').count()

499

In [10]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)


from pyspark.sql.functions import col


search_df.groupBy('searchterm')\
    .count()\
    .orderBy(col('count').desc())\
    .show(5)

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [11]:
# Download model

!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz


--2023-12-11 21:21:42--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 198.23.119.253, 169.45.118.97, ...
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2023-12-11 21:21:43 (242 MB/s) - ‘model.tar.gz’ saved [1490/1490]



In [12]:
# Extract downloaded model

!tar -xzvf model.tar.gz

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet
sales_prediction.model/data/_SUCCESS
sales_prediction.model/data/.part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet.crc
sales_prediction.model/data/._SUCCESS.crc


In [13]:
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml.feature import VectorAssembler

In [14]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

model = LinearRegressionModel.load('sales_prediction.model')

In [15]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    data = [[year, 0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features', 'sales')
    predictions = model.transform(__)
    predictions.select('prediction').show()

In [17]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+

